In [1]:
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
import pyodbc
import sys
import re
import sqlalchemy as sa

In [2]:
agente = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'

url = 'https://www.infomoney.com.br/cotacoes/fundo-imobiliario-cxco11/historico/'

headers = {'User-Agent': agente}

In [3]:
def consultarweb(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        print(response.read())

    except HTTPError as e:
        print(e.status, e.reason)
        print('excessão 1 -> url não localizada!')
    
    except URLError as e:
        print('excessão 2 -> dominio não existe!')
        print(e.reason)



In [4]:
def consultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [5]:
def captura_html_pagina(url):
    html = consultaWebB(url)
    soup = bs(html, 'html.parser')
    return soup

In [6]:
captura_html_pagina(url).prettify()
html = captura_html_pagina(url)
print(html.prettify())

TypeError: object of type 'NoneType' has no len()

In [ ]:
html = captura_html_pagina(url)
print(html.prettify())

In [ ]:
def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic [x]=y
    return dic
        

In [ ]:
def TabelaProposicoes(inicia,quantidade, ano):
    lista = []
    for i in range(quantidade):
        lista = lista + [Conteudo(i+1 +inicia,  ano)]
        print(i+1, 'Consultas efetuadas | aguardando 0.5 segundos')
        time.sleep(0.5)#meio segundo de espaço entre cada iteração da consulta
    tabela = pd.DataFrame(lista)
    pd.options.display.max_colwidth = None
    return tabela

In [ ]:
def Conteudo(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

In [ ]:
def TabelaResultados(inicar_em, quantidade, ano, erros_admissiveis, segundos_espera):
    
    ultima_consulta = inicar_em + quantidade - 1

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while inicar_em <= ultima_consulta and erros <= erros_admissiveis:

        try:
            x = Conteudo(inicar_em,ano)
            lista = lista + [Conteudo(inicar_em,ano)]
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        inicar_em+=1 
        i+=1


    return pd.DataFrame(lista)

In [ ]:
def SQLSelect(query):
    try:
        conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 13 for SQL Server}',
                      server = '(localdb)\MSSQLLocalDB', 
                      database = 'FinalETLProd') 

        out = pd.read_sql_query(query,conn)
        return out
        print('conectado')
    except:
        print('erro ao conectar')

        

In [ ]:
query = '''
    SELECT TOP (10) [Cópia de SubCategoryName]
      ,[Cópia de ProductCategoryID]
      ,[Cópia de ProductSubcategoryID]
      ,[Cópia de ProductID]
      ,[Cópia de ProductName]
      ,[Cópia de ProductNumber]
      ,[Cópia de Color]
      ,[Cópia de ListPrice]
  FROM [FinalETLProd].[dbo].[FinalETLProd]
'''
SQLSelect(query)


In [ ]:
def SQLTruncate(NomeTabela):
        conn = pyodbc.connect('Trusted_Connection=yes', 
                        driver = '{ODBC Driver 17 for SQL Server}',
                        server = '(localdb)\MSSQLLocalDB', 
                        database = 'FinalETLProd') 


        cursor = conn.cursor()

        cursor.execute(f'''

                   TRUNCATE TABLE {NomeTabela}

                   ''')

        conn.commit()
        cursor.close()

In [ ]:
SQLTruncate('informar a tabela')

In [ ]:
def SQLInsertProposicoes(TabelaProposicoes):
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor','Proposicao', 'Ano','Texto'] )
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = '(localdb)\MSSQLLocalDB', 
                          database = 'FinalETLProd')
    
    
    cursor = conn.cursor()
    
    for index, row in TabelaProposicoes.iterrows():
        cursor.execute('''
            insert into Proposicoes(
                 [DataReuniao]
                ,[DataDeliberacao]
                ,[Situacao]
                ,[Assunto]
                ,[Autor]
                ,[Proposicao]
                ,[Ano]
                ,[Texto]
            )
            values(?,?,?,?,?,?,?,?) 
         ''' , # mesma quantidade de variáveis
    
        row['Reunião'], 
        row['Deliberação'], 
        row['Situação'], 
        row['Assunto'], 
        row['Autor'], 
        row['Proposição'], 
        row['Ano'], 
        row['Texto']
    
        )
    
    conn.commit()
    cursor.close()
    

In [ ]:
teste = TabelaResultados(206,100,2021,2,0.5)
#teste #(execulta a consulta)

In [ ]:
SQLInsertProposicoes(teste)


In [ ]:
################---inicio-------#######
proposicao = 203
ano = 2021
dados = Conteudo(proposicao, ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
#======fim======
SQLSelect(f'SELECT * FROM Proposicoes where Proposicao = {proposicao} and ano ={ano}')


In [ ]:
ano=2021
dados=SQLSelect(f'SELECT max(proposicao) proposicao from Proposicoes where ano ={ano}')
dados

In [ ]:
dados['proposicao'].loc[0]

In [ ]:

# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

proxima_proposicao = int(ultima_proposicao) + 1

dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

In [ ]:
# Limpar os dados da tabela só pra demonstração
#SQLTruncate('Proposicoes')


# ========= INICIO =========
proposicao = 203
ano = 2021
dados = Conteudo(proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect(f'select * from Proposicoes where Proposicao = {proposicao} and Ano = {ano}')

In [ ]:
ano = 2021
dados = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
dados

In [ ]:
# acessando a coluna
dados['Proposicao']
dados.Proposicao

In [ ]:
# acessando a coluna e linha
dados['Proposicao'].loc[0]

In [ ]:
# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

proxima_proposicao = int(ultima_proposicao) + 1

dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

In [ ]:
SQLTruncate('Proposicoes')

In [ ]:
# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]


# ----- INICIO ALTERAÇÃO ----- 
if ultima_proposicao == None:
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao) + 1 
# ----- FIM ALTERAÇÃO ----- 


dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

In [ ]:
def InsereProximaProposicao(ano):
    
    # Busca última proposição cadastrada
    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica se foi identificado lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [ ]:
InsereProximaProposicao(1996)
SQLSelect('select *from Proposicoes')

In [ ]:
SQLTruncate('Proposicoes')

In [ ]:
ano = 1996
quantidade = 999990
erros_admissiveis = 2
segundos_espera = 0

# erros
erros = 0
# variaveis do loop
i = 1
lista = []
while erros <= erros_admissiveis:
    try:
        InsereProximaProposicao(ano)
    except:
        erros += 1
        pass
    time.sleep(segundos_espera)
    # carregamento incremental das variáveis
    i += 1


In [ ]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

In [ ]:
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 1):
    
    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis

In [ ]:
BuscaGravaDadosAno(1996,24)

In [ ]:
ano_fim = 2021
ano_ini = 1996
anos = ano_fim - ano_ini + 1
for i in range(anos):
    print(i+ano_ini)

In [ ]:
list(range(1996,1998+1))

In [ ]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

In [ ]:
ano_inicial = 1996
ano_final = 1996

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravação dos dados do ano: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 999999)
    except:
        pass
    
print('Inserção finalizada 😁')